In [6]:
import sys
sys.path.append('/opt/notebooks/polytrader')

In [7]:
from agents.polymarket.polymarket import Polymarket
from agents.trackers.market_tracker import OrderBook
from typing import Dict

In [8]:
p = Polymarket()

In [9]:
game = p.get_market_by_slug('nfl-sf-buf-2024-12-01')
outcome = game.outcomes[0]
# Initialize dictionary to store order books for each outcome
order_books: Dict[str, OrderBook] = {}

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import time
from collections import deque
import numpy as np

In [ ]:
def handle_market_update(outcome: str, message: dict):
    order_book = order_books.get(outcome)
    if order_book is None:
        order_book = OrderBook()
        order_books[outcome] = order_book
    
    if message['event_type'] == 'book':
        order_book.update_from_book(message)
    elif message['event_type'] == 'price_change':
        order_book.update_from_price_change(message)
    elif message['event_type'] == 'last_trade_price':
        order_book.update_last_trade(message)

def create_live_plot(outcome: str, message: dict):
    order_book = order_books.get(outcome)
    if order_book is None:
        return
    # Initialize data storage
    timestamps = deque(maxlen=100)
    best_asks = deque(maxlen=100)
    best_bids = deque(maxlen=100)
    last_trades = deque(maxlen=100)
    
    # Get current best prices
    best_ask = order_book.get_best_ask()
    best_bid = order_book.get_best_bid()
    
    # Update data
    current_time = time.time()
    timestamps.append(current_time)
    
    best_asks.append(best_ask[0] if best_ask else None)
    best_bids.append(best_bid[0] if best_bid else None)
    last_trades.append(order_book.last_trade.price if order_book.last_trade else None)
    
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Convert timestamps to relative time (seconds from start)
    relative_times = np.array(timestamps) - timestamps[0] if timestamps else []
    
    # Plot data
    ax.plot(relative_times, best_asks, 'r-', label='Best Ask', alpha=0.7)
    ax.plot(relative_times, best_bids, 'g-', label='Best Bid', alpha=0.7)
    ax.plot(relative_times, last_trades, 'b.', label='Last Trade', alpha=0.5)
    
    # Customize plot
    ax.set_title(f'Live Order Book - {outcome}')
    ax.set_xlabel('Time (seconds)')
    ax.set_ylabel('Price')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # Add current values as text
    current_ask = best_asks[-1] if best_asks else None
    current_bid = best_bids[-1] if best_bids else None
    current_trade = last_trades[-1] if last_trades else None
    
    text = f'Current Ask: {current_ask:.3f}\n' if current_ask else ''
    text += f'Current Bid: {current_bid:.3f}\n' if current_bid else ''
    text += f'Last Trade: {current_trade:.3f}' if current_trade else ''
    
    ax.text(0.02, 0.98, text,
            transform=ax.transAxes,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Display plot
    clear_output(wait=True)
    display(fig)

In [ ]:
from agents.trackers.market_tracker import MarketChannel
import asyncio
async def track(market_obj):
    # Initialize market channel
    market_channel = MarketChannel(market_obj)

    await market_channel.connect()

    # Add callback for specific assets
    market_channel.add_outcome_callback(outcome, handle_market_update)
    market_channel.add_outcome_callback(outcome, create_live_plot)

    try:
        # Keep connection alive
        while True:
            await asyncio.sleep(1)
    except KeyboardInterrupt:
        await market_channel.close()

In [ ]:
await track(game)